# Telecom Operator Efficiency Analysis 📞📊

**Author:** Luis Chaumer  
**Role:** Data Analyst  

This project analyzes the efficiency of call center operators in a fictional telecom company.  
Using 25,000 simulated call records over 30 days and 20 operators, we will:

- Explore call center performance and customer experience
- Compute key KPIs (AWT, AHT, FCR, abandonment rate, missed calls ratio, CSAT)
- Identify inefficient operators based on data
- Analyze performance by time of day and call reason
- Perform basic statistical tests to compare operator groups
- Provide actionable recommendations for the business

The dataset used in this analysis is: `data/telecom_calls_dataset.csv`.


## 1. Setup: imports and configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

plt.style.use("default")

# Display options
pd.set_option("display.max_columns", 50)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

## 2. Load dataset

In [ ]:
# Adjust the path if needed depending on your folder structure
data_path = "data/telecom_calls_dataset.csv"

df = pd.read_csv(data_path, parse_dates=["timestamp_start", "timestamp_end"])
df.head()

## 3. Initial exploration

In [ ]:
# Shape of the dataset
df.shape

In [ ]:
# Columns and data types
df.info()

In [ ]:
# Quick statistical summary for numeric columns
df.describe()

## 4. Data quality checks

In [ ]:
# Missing values
df.isna().sum()

In [ ]:
# Check for duplicate call IDs
df["call_id"].duplicated().sum()

## 5. Feature engineering

In [ ]:
# Total time from start to end
df["total_time"] = (df["timestamp_end"] - df["timestamp_start"]).dt.total_seconds()

# Extract date and time features
df["date"] = df["timestamp_start"].dt.date
df["day_of_week"] = df["timestamp_start"].dt.day_name()
df["hour"] = df["timestamp_start"].dt.hour

# Flag peak hours (e.g., 10h–13h and 17h–20h)
df["is_peak_hour"] = df["hour"].isin([10, 11, 12, 13, 17, 18, 19, 20])

df.head()

## 6. KPI definitions

We will focus on the following key performance indicators (KPIs):

- **AWT (Average Wait Time):** average `wait_time` for inbound calls  
- **AHT (Average Handle Time):** average `call_duration` for answered calls  
- **Abandonment Rate:** proportion of inbound calls with status = `abandoned`  
- **Missed Calls Ratio:** proportion of inbound calls with status = `missed`  
- **FCR (First Call Resolution Rate):** proportion of answered calls with `resolution = "resolved"`  
- **CSAT (Customer Satisfaction):** average `customer_satisfaction` score  


### 6.1 Overall KPIs (call center level)

In [ ]:
# Focus on inbound calls for many service KPIs
inbound = df[df["call_type"] == "inbound"].copy()

total_calls = len(df)
total_inbound = len(inbound)

awt = inbound["wait_time"].mean()
aht = df.loc[df["status"] == "answered", "call_duration"].mean()

abandonment_rate = (inbound["status"] == "abandoned").mean()
missed_rate = (inbound["status"] == "missed").mean()

answered_calls = df[df["status"] == "answered"]
fcr = (answered_calls["resolution"] == "resolved").mean()

csat = df["customer_satisfaction"].mean()

overall_kpis = pd.Series({
    "Total calls": total_calls,
    "Total inbound calls": total_inbound,
    "AWT (sec)": awt,
    "AHT (sec)": aht,
    "Abandonment rate": abandonment_rate,
    "Missed calls ratio": missed_rate,
    "FCR": fcr,
    "CSAT": csat
})

overall_kpis

### 6.2 Distributions of wait time and call duration

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(inbound["wait_time"], bins=40)
axes[0].set_title("Distribution of wait time (inbound)")
axes[0].set_xlabel("Wait time (seconds)")
axes[0].set_ylabel("Number of calls")

axes[1].hist(df.loc[df["status"] == "answered", "call_duration"], bins=40)
axes[1].set_title("Distribution of call duration (answered calls)")
axes[1].set_xlabel("Call duration (seconds)")
axes[1].set_ylabel("Number of calls")

plt.tight_layout()
plt.show()

## 7. Operator-level performance

In [ ]:
# Aggregate metrics per operator
operator_stats = df.groupby("operator_id").agg(
    total_calls=("call_id", "count"),
    answered_calls=("status", lambda x: (x == "answered").sum()),
    abandoned_calls=("status", lambda x: (x == "abandoned").sum()),
    missed_calls=("status", lambda x: (x == "missed").sum()),
    avg_wait_time=("wait_time", "mean"),
    avg_call_duration=("call_duration", "mean"),
    resolution_rate=("resolution", lambda x: (x == "resolved").mean()),
    csat=("customer_satisfaction", "mean")
)

operator_stats["abandonment_rate"] = operator_stats["abandoned_calls"] / operator_stats["total_calls"]
operator_stats["missed_rate"] = operator_stats["missed_calls"] / operator_stats["total_calls"]

operator_stats.sort_values("total_calls", ascending=False).head(10)

In [ ]:
# Visualize operator productivity (top 15 by total calls)
top_ops = operator_stats.sort_values("total_calls", ascending=False).head(15)

plt.figure(figsize=(10, 4))
plt.bar(top_ops.index.astype(str), top_ops["total_calls"])
plt.title("Top operators by number of calls handled")
plt.xlabel("Operator ID")
plt.ylabel("Total calls")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Visualize resolution rate and CSAT
plt.figure(figsize=(10, 4))
plt.scatter(operator_stats["resolution_rate"], operator_stats["csat"])
plt.title("Operator resolution rate vs CSAT")
plt.xlabel("Resolution rate")
plt.ylabel("Average CSAT")
plt.grid(True)
plt.tight_layout()
plt.show()

### 7.1 Identifying inefficient operators

In [ ]:
# Define simple thresholds for inefficiency
# (These can be refined and justified in the business context)
awt_threshold = operator_stats["avg_wait_time"].quantile(0.75)  # high wait time
res_threshold = operator_stats["resolution_rate"].quantile(0.25)  # low resolution
csat_threshold = operator_stats["csat"].quantile(0.25)  # low satisfaction

operator_stats["is_inefficient"] = (
    (operator_stats["avg_wait_time"] > awt_threshold) &
    (operator_stats["resolution_rate"] < res_threshold) &
    (operator_stats["csat"] < csat_threshold)
)

inefficient_ops = operator_stats[operator_stats["is_inefficient"]]

inefficient_ops

## 8. Statistical testing: efficient vs inefficient operators

In [ ]:
efficient_ops = operator_stats[~operator_stats["is_inefficient"]]
inefficient_ops = operator_stats[operator_stats["is_inefficient"]]

print("Number of efficient operators:", len(efficient_ops))
print("Number of inefficient operators:", len(inefficient_ops))

In [ ]:
# Compare average wait time
t_stat_wait, p_val_wait = stats.ttest_ind(
    efficient_ops["avg_wait_time"],
    inefficient_ops["avg_wait_time"],
    equal_var=False
)

# Compare resolution rate
t_stat_res, p_val_res = stats.ttest_ind(
    efficient_ops["resolution_rate"],
    inefficient_ops["resolution_rate"],
    equal_var=False
)

# Compare CSAT
t_stat_csat, p_val_csat = stats.ttest_ind(
    efficient_ops["csat"],
    inefficient_ops["csat"],
    equal_var=False
)

pd.DataFrame({
    "metric": ["avg_wait_time", "resolution_rate", "csat"],
    "t_stat": [t_stat_wait, t_stat_res, t_stat_csat],
    "p_value": [p_val_wait, p_val_res, p_val_csat]
})

## 9. Time-based analysis

In [ ]:
# Calls by hour of day
calls_per_hour = df.groupby("hour")["call_id"].count()

plt.figure(figsize=(8, 4))
plt.plot(calls_per_hour.index, calls_per_hour.values, marker="o")
plt.title("Number of calls by hour of day")
plt.xlabel("Hour")
plt.ylabel("Number of calls")
plt.xticks(range(8, 21))
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Average wait time by hour
awt_by_hour = inbound.groupby("hour")["wait_time"].mean()

plt.figure(figsize=(8, 4))
plt.plot(awt_by_hour.index, awt_by_hour.values, marker="o")
plt.title("Average wait time by hour (inbound calls)")
plt.xlabel("Hour")
plt.ylabel("AWT (seconds)")
plt.xticks(range(8, 21))
plt.grid(True)
plt.tight_layout()
plt.show()

## 10. Performance by call reason

In [ ]:
reason_stats = df.groupby("reason").agg(
    total_calls=("call_id", "count"),
    avg_wait_time=("wait_time", "mean"),
    avg_call_duration=("call_duration", "mean"),
    resolution_rate=("resolution", lambda x: (x == "resolved").mean()),
    csat=("customer_satisfaction", "mean")
)

reason_stats

In [ ]:
plt.figure(figsize=(8, 4))
plt.bar(reason_stats.index, reason_stats["csat"])
plt.title("CSAT by call reason")
plt.xlabel("Reason")
plt.ylabel("Average CSAT")
plt.tight_layout()
plt.show()

## 11. (Optional) Export figures for README / portfolio

In [ ]:
# Example: create an 'images' folder and save a key chart

import os

os.makedirs("images", exist_ok=True)

plt.figure(figsize=(8, 4))
plt.plot(awt_by_hour.index, awt_by_hour.values, marker="o")
plt.title("Average wait time by hour (inbound calls)")
plt.xlabel("Hour")
plt.ylabel("AWT (seconds)")
plt.xticks(range(8, 21))
plt.grid(True)
plt.tight_layout()
plt.savefig("images/avg_wait_time_by_hour.png", dpi=120)
plt.close()

## 12. Conclusions and recommendations

_Suggested content for your analysis narrative (you can adapt based on actual results):_

- The call center handles **X calls per day on average**, with peak hours between **10h–13h** and **17h–20h**.
- The **Average Wait Time (AWT)** is around **Y seconds**, which is acceptable / could be improved depending on the internal SLA.
- The **Abandonment Rate** and **Missed Calls ratio** indicate that during peak hours the system is under pressure, leading to a higher risk of customer dissatisfaction.
- We identified **N inefficient operators** with significantly higher wait times, lower resolution rates, and lower CSAT scores compared to the rest of the team (supported by statistical tests).
- Certain call reasons (e.g., _technical_) are associated with longer handling times and slightly lower satisfaction, which suggests a need for better scripts, training, or knowledge base improvements.

**Business recommendations:**

1. **Targeted coaching** for inefficient operators, focusing on resolution skills and call handling structure.
2. **Staffing adjustments during peak hours** to reduce wait time and abandonment.
3. **Process and script review** for the call reasons with the lowest CSAT and longest duration (e.g., technical issues).
4. **Continuous monitoring** via a dashboard (Tableau / Power BI) using the KPIs defined in this analysis.
5. Optionally, implement an **alert system** when AWT or abandonment exceeds a given threshold in real time.

These insights can be summarized visually in a dashboard and included in your portfolio README and presentation.
